In [30]:
from pathlib import Path

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import load_component
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline


In [47]:
credential = DefaultAzureCredential()
credential.get_token("https://management.azure.com/.default")

ml_client = MLClient.from_config(credential=credential)


Found the config file in: /config.json


In [110]:
!source prep-srcs.sh


In [117]:
component_yaml_dir = Path("components")

data_prep = load_component(source = component_yaml_dir / 'data_prep' / "data_prep.yml")
train_kernelridge = load_component(source = component_yaml_dir / 'train_kernelridge' / "train_kernelridge.yml")


In [120]:
@pipeline()
def house_prices_pipeline(data_dir, kernel):

    data_prep_result = data_prep(data_dir = data_dir)
    prep_data_dir = data_prep_result.outputs.prep_data_dir

    train_kernelridge_result = train_kernelridge(prep_data_dir = prep_data_dir, kernel = kernel)

    return {
        "prep_data_dir": data_prep_result.outputs.prep_data_dir,
        "results_dir": train_kernelridge_result.outputs.results_dir
    }


In [121]:
data_dir = Input(type=AssetTypes.URI_FOLDER, path="azureml:data-dir:1")
kernel = 'linear'

pipeline_job = house_prices_pipeline(data_dir = data_dir,
                                     kernel = kernel)


In [122]:
pipeline_job.outputs.prep_data_dir.mode = 'upload'
pipeline_job.outputs.results_dir.mode = 'upload'

pipeline_job.settings.default_compute = "aml-cluster"
pipeline_job.settings.default_datastore = "workspaceblobstore"


In [123]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_house_prices"
)
pipeline_job


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
pipeline_house_prices,tidy_root_1z74zpttwj,pipeline,NotStarted,Link to Azure Machine Learning studio
